# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\n0170199\Projects\DE_Projects\project_2_walkthrough


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
     CREATE KEYSPACE IF NOT EXISTS events
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [7]:
session.set_keyspace('events') 


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
#Table for filtering by sessionId and itemInSession for that reason made the primary keys sessionId and itemInSession.

query = "CREATE TABLE IF NOT EXISTS music_by_session"
query = query + """(artist TEXT,
songTitle TEXT,
songLength FLOAT,
sessionId INT,
itemInSession INT,
PRIMARY KEY (sessionId, itemInSession))"""

print(query)
session.execute(query)

                    

CREATE TABLE IF NOT EXISTS music_by_session(artist TEXT,
songTitle TEXT,
songLength FLOAT,
sessionId INT,
itemInSession INT,
PRIMARY KEY (sessionId, itemInSession))


In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO music_by_session (artist, songTitle, songLength, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"

        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], 
                                line[9],
                                float(line[5]),
                                int(line[8]),
                                int(line[3])
                                ))

In [10]:
#Table for filtering by music by userId and sessionID for that reason made the primary keys userId and sessionID.  Clustering by itemInSession in order to sort

query = "CREATE TABLE IF NOT EXISTS music_by_user_session"
query = query + """(
                    artist TEXT,
                    song TEXT,
                    user TEXT,
                    sessionId INT,
                    userId INT,
                    itemInSession INT,
                    PRIMARY KEY ((userId, sessionId),itemInSession))"""

print(query)
session.execute(query)

                    

CREATE TABLE IF NOT EXISTS music_by_user_session(
                    artist TEXT,
                    song TEXT,
                    user TEXT,
                    sessionId INT,
                    userId INT,
                    itemInSession INT,
                    PRIMARY KEY ((userId, sessionId),itemInSession))


In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO music_by_user_session (artist, song, user, sessionId, userId, itemInSession)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s)"

        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        #Created user column to join first and last name together in one column.
        session.execute(query, (line[0], 
                                line[9],
                                line[1] + ' ' + line[4],
                                int(line[8]),
                                int(line[10]),
                                int(line[3]),
                                ))

In [12]:
#Table for filtering music by "All Hands against his own" but wanting to return the song and user.  Primary keys set to song and user 

query = "CREATE TABLE IF NOT EXISTS music_all_hands_song"
query = query + """(
                    song TEXT,
                    user TEXT,
                    PRIMARY KEY ( song, user ))"""

print(query)
session.execute(query)

                    

CREATE TABLE IF NOT EXISTS music_all_hands_song(
                    song TEXT,
                    user TEXT,
                    PRIMARY KEY ( song, user ))


In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO music_all_hands_song (song, user)"
        query = query + "VALUES (%s,%s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        #Created user column to join first and last name together in one column.
        session.execute(query, (line[9], line[1] + ' ' + line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
##test results against music_by_session
query = """SELECT artist,
                songTitle,
                songLength,
                sessionId,
                itemInSession
            FROM  music_by_session"""

try:
    rows = session.execute(query)
    print("query successful")
except Exception as e: 
    print(e)

##test results against music_by_user_session
query = """ SELECT artist,
                    song,
                    user,
                    sessionId,
                    userId,
                    itemInSession
            FROM music_by_user_session """

try:
    rows = session.execute(query)
    print("query successful")
except Exception as e: 
    print(e)

##test results against music_all_hands_song

query = """SELECT song, user
FROM music_all_hands_song """

try:
    rows = session.execute(query)
    print("query successful")
except Exception as e: 
    print(e)

query successful
query successful
query successful


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [29]:
## queryfor sessionId = 338 and itemInSession = 4

query = """select artist, songtitle, songlength
from music_by_session
WHERE sessionId = 338 and itemInSession = 4 """

try:
    rows = session.execute(query)
except Exception as e: 
    print(e)
    
for row in rows:
    print(row.artist, row.songtitle,  row.songlength)

                    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [24]:
## Query for userid = 10, sessionid = 182
query = """select artist, song, user
from music_by_user_session
WHERE userid = 10 and sessionid = 182 """

try:
    rows = session.execute(query)
except Exception as e: 
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.user)

                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [25]:
## Get all users who listened to the song 'All Hands Against His Own'

query = """select user
from music_all_hands_song
WHERE song = 'All Hands Against His Own' """

try:
    rows = session.execute(query)
except Exception as e: 
    print(e)
    
for row in rows:
    print(row.user)


                    

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [30]:
session.execute("DROP TABLE music_by_session")

In [31]:
session.execute("DROP TABLE music_by_user_session")

In [32]:
session.execute("DROP TABLE music_all_hands_song")

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()